In [0]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Import necessary libraries
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import cv2
import os

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Read recipe inputs
vid_files_s3 = dataiku.Folder("R0VYPLS0")
vid_files_fs = dataiku.Folder("Go6vQdUp")

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Write recipe outputs
vid_fps_s3 = dataiku.Folder("l11wPapI")

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Define helper function to create frames from video files
def getFrame(filename, sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(f"image{str(count)}.jpg", image) # save frame as JPG file
        vid_fps_s3.upload_file(f"{filename}/image{str(count)}.jpg", f"image{str(count)}.jpg")
        os.remove(f"image{str(count)}.jpg")
    return hasFrames

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Loop through each video file in path to create frames for video
for mp4_filename in vid_files_s3.list_paths_in_partition():
    with vid_files_s3.get_download_stream(mp4_filename) as stream:
        # Upload to local folder
        vid_files_fs.upload_stream(mp4_filename, stream)

        # Create opencv object
        vidcap = cv2.VideoCapture(os.path.join(vid_files_fs.get_path(), mp4_filename[1:]))

        # Initialise settings to create frames on video
        sec = 0
        frameRate = 5 #//it will capture image in each 5 second
        count=1
        filename = mp4_filename.split(".")[0][1:]
        success = getFrame(filename, sec)
        while success:
            count = count + 1
            sec = sec + frameRate
            sec = round(sec, 2)
            success = getFrame(filename, sec)

        vidcap.release()

        vid_files_fs.clear()